In [1]:
import os
import torch
import skimage
import pywt
import scipy.io
import scipy.signal
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from scipy import stats
from einops import reduce, rearrange, repeat
from npeet import entropy_estimators as ee
from torch.optim.lr_scheduler import StepLR
from scipy.fft import rfft, rfftfreq, ifft
from einops import rearrange
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from Electrodes import Electrodes
from tqdm import tqdm

In [2]:
class DEAPDatasetEEGFeatures(InMemoryDataset):
  def __init__(self, root, raw_dir, processed_dir, feature='de',target='participant_id', transform=None, pre_transform=None,include_edge_attr = True, undirected_graphs = True, add_global_connections=True, participant_from=1, participant_to=32, n_videos=40):
      self._raw_dir = raw_dir
      self._processed_dir = processed_dir
      self.participant_from = participant_from
      self.participant_to = participant_to
      self.n_videos = n_videos
      self.feature = feature
      self.target = target
      # Whether or not to include edge_attr in the dataset
      self.include_edge_attr = include_edge_attr
      # If true there will be 1024 links as opposed to 528
      self.undirected_graphs = undirected_graphs
      # Instantiate class to handle electrode positions
      print('Using global connections' if add_global_connections else 'Not using global connections')
      self.electrodes = Electrodes(add_global_connections, expand_3d = False)
      super(DEAPDatasetEEGFeatures, self).__init__(root, transform, pre_transform)
      self.data, self.slices = torch.load(self.processed_paths[0])
      
  @property
  def raw_dir(self):
      return f'{self.root}/{self._raw_dir}'

  @property
  def processed_dir(self):
      return f'{self.root}/{self._processed_dir}'

  @property
  def raw_file_names(self):
      raw_names = [f for f in os.listdir(self.raw_dir)]
      raw_names.sort()
      return raw_names

  @property
  def processed_file_names(self):
      if not os.path.exists(self.processed_dir):
        os.makedirs(self.processed_dir)
      file_name = f'{self.participant_from}-{self.participant_to}' if self.participant_from is not self.participant_to else f'{self.participant_from}'
      return [f'deap_processed_graph.{file_name}_{self.feature}_{self.target}.dataset']

  def process(self):
        # Number of nodes per graph
        n_nodes = len(self.electrodes.positions_3d)
        

        if self.undirected_graphs:
            source_nodes, target_nodes = np.repeat(np.arange(0,n_nodes),n_nodes), np.tile(np.arange(0,n_nodes),n_nodes)
        else:
            source_nodes, target_nodes = np.tril_indices(n_nodes,n_nodes)
        
        edge_attr = self.electrodes.adjacency_matrix[source_nodes,target_nodes]
        
        # Remove zero weight links
        mask = np.ma.masked_not_equal(edge_attr, 0).mask
        edge_attr,source_nodes,target_nodes = edge_attr[mask], source_nodes[mask], target_nodes[mask]

        edge_attr, edge_index = torch.FloatTensor(edge_attr), torch.tensor([source_nodes,target_nodes], dtype=torch.long)
        
        # Expand edge_index and edge_attr to match windows
        e_edge_index = edge_index.clone()
        e_edge_attr = edge_attr.clone()
        
        number_of_graphs = 5
        for i in range(number_of_graphs-1):
            a = edge_index + e_edge_index.max() + 1
            e_edge_index = torch.cat([e_edge_index,a],dim=1)
            e_edge_attr = torch.cat([e_edge_attr,edge_attr],dim=0)

        print(f'Number of graphs per video: {number_of_graphs}')
        # List of graphs that will be written to file
        data_list = []
        pbar = tqdm(range(self.participant_from,self.participant_to+1))
        for participant_id in pbar:
            raw_name = [e for e in self.raw_file_names if str(participant_id).zfill(2) in e][0]
            pbar.set_description(raw_name)
            # Load raw file as np array
            participant_data = scipy.io.loadmat(f'{self.raw_dir}/{raw_name}')
            signal_data = torch.FloatTensor(remove_baseline_mean(participant_data['data'][:,:32,:]))
#             signal_data = torch.FloatTensor()
            processed = []
            for i, video in enumerate(signal_data[:self.n_videos,:,:]):
                if self.feature == 'wav':
                    node_features = process_video_wavelet(video)
                else:
                    node_features = process_video(video, feature=self.feature)
                
                if self.target == 'emotion_labels':
                    target = [participant_data['labels'][i]]
                if self.target == 'participant_id':
                    target = participant_id-1
                elif self.target == 'video_id':
                    pass
                else:
                    raise 'Invalid target'
                data = Data(x=torch.FloatTensor(node_features),edge_attr=e_edge_attr,edge_index=e_edge_index, y=torch.LongTensor([target])) if self.include_edge_attr else Data(x=torch.FloatTensor(node_features), edge_index=e_edge_index, y=torch.LongTensor([target]))
                data_list.append(data) 
               
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [3]:
def calculate_de(window):
    return ee.entropy(window.reshape(-1,1), k=2)
# Input: Video with shape (32,7680)
# Output: Graph node features with shape (5*32, 59) -> 5 graphs with 32 nodes each with 59 features each
def process_video(video, feature='psd'):
    # Transform to frequency domain
    fft_vals = np.fft.rfft(video, axis=-1)
     # Get frequencies for amplitudes in Hz
    samplingFrequency = 128
    fft_freq = np.fft.rfftfreq(video.shape[-1], 1.0/samplingFrequency)
    # Delta, Theta, Alpha, Beta, Gamma
    bands = [(0,4),(4,8),(8,12),(12,30),(30,45)]
    
    band_mask = np.array([np.logical_or(fft_freq < f, fft_freq > t) for f,t in bands])
    band_mask = repeat(band_mask,'a b -> a c b', c=32)
    band_data = np.array(fft_vals)
    band_data = repeat(band_data,'a b -> c a b', c=5)
     
    band_data[band_mask] = 0
    
    band_data = np.fft.irfft(band_data)

    windows = skimage.util.view_as_windows(band_data, (5,32,128), step=128).squeeze()
    # (5, 32, 60, 128)
    windows = rearrange(windows, 'a b c d -> b c a d')
    
    if feature == 'psd':
        features = scipy.signal.periodogram(windows)[1]
        features = np.mean(features, axis=-1)
    elif feature == 'de':
        features = np.apply_along_axis(calculate_de, -1, windows)

    
    features = rearrange(features, 'a b c -> (a b) c')
    features = torch.FloatTensor(features)

    return features

In [4]:
def remove_baseline_mean(signal_data):
    # Take first three senconds of data
    signal_baseline = np.array(signal_data[:,:,:128*3]).reshape(40,32,128,-1)
    # Mean of three senconds of baseline will be deducted from all windows
    signal_noise = np.mean(signal_baseline,axis=-1)
    # Expand mask
    signal_noise = repeat(signal_noise,'a b c -> a b (d c)',d=60)
    return signal_data[:,:,128*3:] - signal_noise

In [5]:
def process_video_wavelet(video, feature='energy', time_domain=False):
    band_widths = [32,16,8,4]
    features = []
    for i in range(5):
        if i == 0:
            # Highest frequencies (64-128Hz) are not used
            cA, cD = pywt.dwt(video.numpy(), 'db4')
        else:
            cA, cD = pywt.dwt(cA, 'db4')
            
            cA_windows = skimage.util.view_as_windows(cA, (32,band_widths[i-1]*2), step=band_widths[i-1]).squeeze()
            cA_windows = np.transpose(cA_windows[:59,:,:],(1,0,2))
            if feature == 'energy':
                cA_windows = np.square(cA_windows)
                cA_windows = np.sum(cA_windows, axis=-1)
                features.append(cA_windows)
                
    if time_domain:
        features = np.transpose(features,(2,1,0))
    features = rearrange(features, 'a b c -> (a b) c')
    features = torch.FloatTensor(features)
    
    # Normalization
    m = features.mean(0, keepdim=True)
    s = features.std(0, unbiased=False, keepdim=True)
    features -= m
    features /= s
    return features

In [6]:
# Constants used to define data paths
ROOT_DIR = './'
RAW_DIR = 'data/matlabPREPROCESSED'
PROCESSED_DIR = 'data/graphProcessedData'

dataset = DEAPDatasetEEGFeatures(root= ROOT_DIR, raw_dir= RAW_DIR, processed_dir= PROCESSED_DIR, feature='psd',participant_to=32)
# dataset = dataset.shuffle()

s01.mat:   0%|          | 0/32 [00:00<?, ?it/s]

Using global connections
Processing...
Number of graphs per video: 5


s32.mat: 100%|██████████| 32/32 [01:09<00:00,  2.16s/it]

Done!


In [7]:
# 880 used for training, 220 validation and 180 testing
# test_participant = 1
# 
splt_idx = 1100

dataset = dataset.shuffle()

# 85% used for train/val
train_dataset = dataset[:splt_idx]
test_dataset = dataset[splt_idx:]

len(train_dataset),len(test_dataset)

(1100, 180)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(device)

cuda


In [17]:
from torch_geometric.nn import GCN2Conv, GCNConv, global_max_pool as gmp
class Model(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels=128, n_classes = 32):
        super(Model, self).__init__()
        
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        
        self.gconv1 = GCNConv(in_channels,hidden_channels*2)
        self.gconv2 = GCNConv(hidden_channels*2,hidden_channels)
        
#         self.gconv3 = GCNConv(in_channels,hidden_channels)
        
        # self.rnn = torch.nn.GRU(hidden_channels, rnn_hidden_dim, 2,dropout=0.2, batch_first=True)
        self.cnn1 = torch.nn.Conv1d(5, 1, kernel_size=1, stride=1)
        
        self.lin1 = torch.nn.Linear(32*hidden_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, n_classes)

        self.softmax = nn.Softmax(dim=-1)

        
    def forward(self, batch):
        bs = len(torch.unique(batch.batch))
        x, edge_index, edge_attr = batch.x, batch.edge_index, batch.edge_attr
#         print(x.shape)
        x = self.gconv1(x, edge_index, edge_attr)
        x = self.gconv2(x, edge_index, edge_attr)
        x = F.dropout(x, p=0.4, training=self.training)
        x = x.relu()
#         print(x.shape)
        x = rearrange(x, '(bs g e) f -> (bs e) g f', bs=bs, e=32)
#         print(x.shape)
        x = self.cnn1(x).squeeze()
        x = x.tanh()
        x = rearrange(x, '(bs e) f -> bs (e f)', bs=bs)
#         print(x.shape)
#         x = torch.sum(x, dim=1)
        
        
        x = F.dropout(x, p=0.4, training=self.training)
#         print(x.shape)
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
#         print(x.shape)
#         x = x.view(-1)
       
        x = self.softmax(x)
        return x

        

In [ ]:
# %%timeit

model = Model(train_dataset[0].x.shape[1]).to(device)  
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f'Model parameter count: {pytorch_total_params}')

# model = model.to(devic)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=.1, rho=0.9, eps=1e-06, weight_decay=1e-5)
# optimizer = torch.optim.SGD(model.parameters(),lr=1e-5, weight_decay=1e-4)
# optimizer = torch.optim.Adam(model.parameters(),lr=1e-4, weight_decay=1e-2)
# optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-5)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=5e-4)
optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-4, lr_decay=0, weight_decay=0)

# Instantiate optimizer
# scheduler = StepLR(optimizer, step_size=20, gamma=0.7)

# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()



def train(loader, target = 0):
    model.train()
    losses = []
    right = 0
    tot = 0
    for batch in loader:
        optimizer.zero_grad()
        batch = batch.to(device)
        out = model(batch)
        loss = criterion(out,batch.y)
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        pred = torch.argmax(out,-1)
        right += torch.sum((pred == batch.y).detach().cpu())
        tot += batch.y.shape[0]
        
    return np.array(losses).mean(), right/tot

def test(loader,verbose=False, target = 0):
    model.eval()
    losses = []
    right = 0
    tot = 0
    for batch in loader:
        batch = batch.to(device)
        # y = batch.y[:,target]
        out = model(batch)
        pred = torch.argmax(out,-1)
        if verbose:
            print(pred,batch.y)
        loss = criterion(out,batch.y)
        losses.append(loss.item())
        
        right += torch.sum((pred == batch.y).detach().cpu())
        tot += batch.y.shape[0]
    return np.array(losses).mean(), right/tot

best_val_loss = np.inf
esp = 0
MAX_ESP = 50

BS = 8

target = 0 # Valence-Arousal-Dominance-Liking

splt_idx = 1000
train_data, val_data = torch.utils.data.random_split(train_dataset, [splt_idx, len(train_dataset)-splt_idx])

train_loader = DataLoader(train_data, batch_size=BS, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BS)
for epoch in range(1, 10000):    

    # Training and validation
    train_loss, train_acc = train(train_loader, target = target)
    val_loss, val_acc = test(val_loader , target = target)
    print(f'Epoch {epoch};t loss: {train_loss:.5f} ;t acc: {train_acc:.2f} ;v loss: {val_loss:.5f} ;v acc: {val_acc:.2f}')

    # Early stopping and checkpoint
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        esp = 0
        torch.save(model.state_dict(),'./best_params') 
    else:
        esp += 1
        if esp >= MAX_ESP:
            break
            
    if epoch % 20 == 0:
        test_loader = DataLoader(test_dataset, batch_size=2)
        loss, acc = test(test_loader, True)
        print(f'Test loss: {loss} ; Test acc: {acc}')


print('Finished training')

Model parameter count: 577062
Epoch 1;t loss: 3.46500 ;t acc: 0.03 ;v loss: 3.46355 ;v acc: 0.06
Epoch 2;t loss: 3.46380 ;t acc: 0.05 ;v loss: 3.46210 ;v acc: 0.09
Epoch 3;t loss: 3.46286 ;t acc: 0.07 ;v loss: 3.46101 ;v acc: 0.10
Epoch 4;t loss: 3.46248 ;t acc: 0.07 ;v loss: 3.46001 ;v acc: 0.13
Epoch 5;t loss: 3.46184 ;t acc: 0.07 ;v loss: 3.45909 ;v acc: 0.14
Epoch 6;t loss: 3.46123 ;t acc: 0.09 ;v loss: 3.45819 ;v acc: 0.14
Epoch 7;t loss: 3.46111 ;t acc: 0.08 ;v loss: 3.45729 ;v acc: 0.14
Epoch 8;t loss: 3.45964 ;t acc: 0.10 ;v loss: 3.45625 ;v acc: 0.15
Epoch 9;t loss: 3.46021 ;t acc: 0.09 ;v loss: 3.45528 ;v acc: 0.16
Epoch 10;t loss: 3.45888 ;t acc: 0.11 ;v loss: 3.45428 ;v acc: 0.15
Epoch 11;t loss: 3.45818 ;t acc: 0.10 ;v loss: 3.45333 ;v acc: 0.16
Epoch 12;t loss: 3.45793 ;t acc: 0.12 ;v loss: 3.45240 ;v acc: 0.16
Epoch 13;t loss: 3.45769 ;t acc: 0.11 ;v loss: 3.45137 ;v acc: 0.16
Epoch 14;t loss: 3.45699 ;t acc: 0.12 ;v loss: 3.45030 ;v acc: 0.15
Epoch 15;t loss: 3.45667 ;t

Epoch 32;t loss: 3.44500 ;t acc: 0.13 ;v loss: 3.42565 ;v acc: 0.18
Epoch 33;t loss: 3.44480 ;t acc: 0.13 ;v loss: 3.42416 ;v acc: 0.18
Epoch 34;t loss: 3.44256 ;t acc: 0.14 ;v loss: 3.42260 ;v acc: 0.18
Epoch 35;t loss: 3.44224 ;t acc: 0.14 ;v loss: 3.42113 ;v acc: 0.18
Epoch 36;t loss: 3.44157 ;t acc: 0.13 ;v loss: 3.41974 ;v acc: 0.18
Epoch 37;t loss: 3.44107 ;t acc: 0.14 ;v loss: 3.41818 ;v acc: 0.19
Epoch 38;t loss: 3.44003 ;t acc: 0.13 ;v loss: 3.41692 ;v acc: 0.18
Epoch 39;t loss: 3.43930 ;t acc: 0.14 ;v loss: 3.41551 ;v acc: 0.17
Epoch 40;t loss: 3.43777 ;t acc: 0.14 ;v loss: 3.41412 ;v acc: 0.17
tensor([23, 23], device='cuda:0') tensor([21,  8], device='cuda:0')
tensor([9, 9], device='cuda:0') tensor([ 3, 28], device='cuda:0')
tensor([23,  6], device='cuda:0') tensor([31, 12], device='cuda:0')
tensor([10,  6], device='cuda:0') tensor([10,  6], device='cuda:0')
tensor([21, 10], device='cuda:0') tensor([28,  1], device='cuda:0')
tensor([ 6, 21], device='cuda:0') tensor([ 7, 16],

tensor([10,  6], device='cuda:0') tensor([10, 13], device='cuda:0')
tensor([21,  6], device='cuda:0') tensor([24, 27], device='cuda:0')
tensor([10,  9], device='cuda:0') tensor([3, 1], device='cuda:0')
tensor([6, 6], device='cuda:0') tensor([31, 12], device='cuda:0')
tensor([10, 21], device='cuda:0') tensor([10, 19], device='cuda:0')
tensor([ 6, 23], device='cuda:0') tensor([30, 31], device='cuda:0')
tensor([15, 23], device='cuda:0') tensor([15, 23], device='cuda:0')
tensor([10, 21], device='cuda:0') tensor([13, 24], device='cuda:0')
tensor([21,  6], device='cuda:0') tensor([21, 11], device='cuda:0')
tensor([10, 23], device='cuda:0') tensor([ 8, 23], device='cuda:0')
tensor([6, 6], device='cuda:0') tensor([30, 24], device='cuda:0')
tensor([ 6, 21], device='cuda:0') tensor([30,  4], device='cuda:0')
tensor([ 6, 21], device='cuda:0') tensor([ 7, 21], device='cuda:0')
tensor([9, 9], device='cuda:0') tensor([ 3, 30], device='cuda:0')
tensor([23,  6], device='cuda:0') tensor([18,  8], devic

Epoch 81;t loss: 3.40150 ;t acc: 0.17 ;v loss: 3.36760 ;v acc: 0.15
Epoch 82;t loss: 3.40013 ;t acc: 0.17 ;v loss: 3.36668 ;v acc: 0.15
Epoch 83;t loss: 3.40030 ;t acc: 0.17 ;v loss: 3.36555 ;v acc: 0.15
Epoch 84;t loss: 3.39965 ;t acc: 0.17 ;v loss: 3.36455 ;v acc: 0.15
Epoch 85;t loss: 3.39799 ;t acc: 0.18 ;v loss: 3.36362 ;v acc: 0.16
Epoch 86;t loss: 3.39676 ;t acc: 0.18 ;v loss: 3.36278 ;v acc: 0.15
Epoch 87;t loss: 3.39562 ;t acc: 0.17 ;v loss: 3.36182 ;v acc: 0.15
Epoch 88;t loss: 3.39375 ;t acc: 0.19 ;v loss: 3.36088 ;v acc: 0.15
Epoch 89;t loss: 3.39457 ;t acc: 0.18 ;v loss: 3.36004 ;v acc: 0.15
Epoch 90;t loss: 3.39415 ;t acc: 0.17 ;v loss: 3.35914 ;v acc: 0.15
Epoch 91;t loss: 3.39251 ;t acc: 0.18 ;v loss: 3.35826 ;v acc: 0.15
Epoch 92;t loss: 3.39207 ;t acc: 0.19 ;v loss: 3.35747 ;v acc: 0.15
Epoch 93;t loss: 3.39180 ;t acc: 0.19 ;v loss: 3.35658 ;v acc: 0.15
Epoch 94;t loss: 3.39045 ;t acc: 0.18 ;v loss: 3.35571 ;v acc: 0.15
Epoch 95;t loss: 3.38983 ;t acc: 0.19 ;v loss: 3

Epoch 111;t loss: 3.37588 ;t acc: 0.20 ;v loss: 3.34443 ;v acc: 0.17
Epoch 112;t loss: 3.37642 ;t acc: 0.21 ;v loss: 3.34389 ;v acc: 0.17
Epoch 113;t loss: 3.37600 ;t acc: 0.20 ;v loss: 3.34330 ;v acc: 0.17
Epoch 114;t loss: 3.37364 ;t acc: 0.20 ;v loss: 3.34269 ;v acc: 0.17
Epoch 115;t loss: 3.37469 ;t acc: 0.20 ;v loss: 3.34213 ;v acc: 0.17
Epoch 116;t loss: 3.37338 ;t acc: 0.20 ;v loss: 3.34167 ;v acc: 0.17
Epoch 117;t loss: 3.37322 ;t acc: 0.21 ;v loss: 3.34136 ;v acc: 0.17
Epoch 118;t loss: 3.37269 ;t acc: 0.20 ;v loss: 3.34085 ;v acc: 0.17
Epoch 119;t loss: 3.37228 ;t acc: 0.21 ;v loss: 3.34042 ;v acc: 0.17
Epoch 120;t loss: 3.36987 ;t acc: 0.20 ;v loss: 3.33992 ;v acc: 0.17
tensor([21, 21], device='cuda:0') tensor([21,  8], device='cuda:0')
tensor([23, 21], device='cuda:0') tensor([ 3, 28], device='cuda:0')
tensor([21,  6], device='cuda:0') tensor([31, 12], device='cuda:0')
tensor([10,  6], device='cuda:0') tensor([10,  6], device='cuda:0')
tensor([21, 10], device='cuda:0') tens

...

Epoch 4621;t loss: 2.93201 ;t acc: 0.64 ;v loss: 3.08311 ;v acc: 0.45
Epoch 4622;t loss: 2.92710 ;t acc: 0.64 ;v loss: 3.08308 ;v acc: 0.45
Epoch 4623;t loss: 2.93134 ;t acc: 0.64 ;v loss: 3.08302 ;v acc: 0.45
Epoch 4624;t loss: 2.93169 ;t acc: 0.64 ;v loss: 3.08299 ;v acc: 0.45
Epoch 4625;t loss: 2.93222 ;t acc: 0.64 ;v loss: 3.08296 ;v acc: 0.45
Epoch 4626;t loss: 2.93178 ;t acc: 0.64 ;v loss: 3.08296 ;v acc: 0.45
Epoch 4627;t loss: 2.93118 ;t acc: 0.65 ;v loss: 3.08292 ;v acc: 0.45
Epoch 4628;t loss: 2.92980 ;t acc: 0.64 ;v loss: 3.08291 ;v acc: 0.45
Epoch 4629;t loss: 2.93612 ;t acc: 0.64 ;v loss: 3.08289 ;v acc: 0.45
Epoch 4630;t loss: 2.92622 ;t acc: 0.64 ;v loss: 3.08284 ;v acc: 0.45
Epoch 4631;t loss: 2.92949 ;t acc: 0.64 ;v loss: 3.08283 ;v acc: 0.45
Epoch 4632;t loss: 2.92917 ;t acc: 0.65 ;v loss: 3.08279 ;v acc: 0.45
Epoch 4633;t loss: 2.92863 ;t acc: 0.64 ;v loss: 3.08282 ;v acc: 0.45
Epoch 4634;t loss: 2.93131 ;t acc: 0.64 ;v loss: 3.08280 ;v acc: 0.45
Epoch 4635;t loss: 2

Epoch 4651;t loss: 2.92650 ;t acc: 0.64 ;v loss: 3.08233 ;v acc: 0.45
Epoch 4652;t loss: 2.93164 ;t acc: 0.64 ;v loss: 3.08229 ;v acc: 0.45
Epoch 4653;t loss: 2.93055 ;t acc: 0.64 ;v loss: 3.08229 ;v acc: 0.45
Epoch 4654;t loss: 2.92856 ;t acc: 0.64 ;v loss: 3.08223 ;v acc: 0.45
Epoch 4655;t loss: 2.93013 ;t acc: 0.65 ;v loss: 3.08222 ;v acc: 0.45
Epoch 4656;t loss: 2.93044 ;t acc: 0.64 ;v loss: 3.08218 ;v acc: 0.45
Epoch 4657;t loss: 2.93401 ;t acc: 0.64 ;v loss: 3.08215 ;v acc: 0.45
Epoch 4658;t loss: 2.93220 ;t acc: 0.64 ;v loss: 3.08213 ;v acc: 0.45
Epoch 4659;t loss: 2.92681 ;t acc: 0.64 ;v loss: 3.08210 ;v acc: 0.45
Epoch 4660;t loss: 2.92692 ;t acc: 0.65 ;v loss: 3.08206 ;v acc: 0.45
tensor([21,  8], device='cuda:0') tensor([21,  8], device='cuda:0')
tensor([ 2, 28], device='cuda:0') tensor([ 3, 28], device='cuda:0')
tensor([ 4, 12], device='cuda:0') tensor([31, 12], device='cuda:0')
tensor([10,  6], device='cuda:0') tensor([10,  6], device='cuda:0')
tensor([28, 10], device='cud

tensor([ 5, 12], device='cuda:0') tensor([16, 20], device='cuda:0')
tensor([ 0, 16], device='cuda:0') tensor([ 0, 16], device='cuda:0')
tensor([ 2, 15], device='cuda:0') tensor([ 2, 15], device='cuda:0')
tensor([10,  6], device='cuda:0') tensor([11, 26], device='cuda:0')
tensor([5, 2], device='cuda:0') tensor([ 5, 26], device='cuda:0')
tensor([12, 12], device='cuda:0') tensor([ 2, 30], device='cuda:0')
tensor([ 9, 12], device='cuda:0') tensor([30, 12], device='cuda:0')
tensor([ 5, 18], device='cuda:0') tensor([ 5, 18], device='cuda:0')
tensor([14, 12], device='cuda:0') tensor([23, 13], device='cuda:0')
tensor([10,  6], device='cuda:0') tensor([19,  6], device='cuda:0')
tensor([10, 12], device='cuda:0') tensor([10, 13], device='cuda:0')
tensor([ 4, 27], device='cuda:0') tensor([24, 27], device='cuda:0')
tensor([5, 1], device='cuda:0') tensor([3, 1], device='cuda:0')
tensor([12, 12], device='cuda:0') tensor([31, 12], device='cuda:0')
tensor([10,  2], device='cuda:0') tensor([10, 19], dev

Epoch 4701;t loss: 2.92896 ;t acc: 0.64 ;v loss: 3.08118 ;v acc: 0.45
Epoch 4702;t loss: 2.92746 ;t acc: 0.64 ;v loss: 3.08112 ;v acc: 0.45
Epoch 4703;t loss: 2.92556 ;t acc: 0.64 ;v loss: 3.08108 ;v acc: 0.45
Epoch 4704;t loss: 2.92956 ;t acc: 0.64 ;v loss: 3.08103 ;v acc: 0.45
Epoch 4705;t loss: 2.93034 ;t acc: 0.65 ;v loss: 3.08103 ;v acc: 0.45
Epoch 4706;t loss: 2.92770 ;t acc: 0.64 ;v loss: 3.08100 ;v acc: 0.45
Epoch 4707;t loss: 2.93220 ;t acc: 0.64 ;v loss: 3.08100 ;v acc: 0.45
Epoch 4708;t loss: 2.92624 ;t acc: 0.64 ;v loss: 3.08099 ;v acc: 0.45
Epoch 4709;t loss: 2.92835 ;t acc: 0.65 ;v loss: 3.08094 ;v acc: 0.45
Epoch 4710;t loss: 2.92649 ;t acc: 0.64 ;v loss: 3.08090 ;v acc: 0.45
Epoch 4711;t loss: 2.92864 ;t acc: 0.64 ;v loss: 3.08088 ;v acc: 0.45
Epoch 4712;t loss: 2.92677 ;t acc: 0.64 ;v loss: 3.08087 ;v acc: 0.45
Epoch 4713;t loss: 2.92901 ;t acc: 0.64 ;v loss: 3.08086 ;v acc: 0.45
Epoch 4714;t loss: 2.92825 ;t acc: 0.64 ;v loss: 3.08083 ;v acc: 0.45
Epoch 4715;t loss: 2

Epoch 4731;t loss: 2.92737 ;t acc: 0.64 ;v loss: 3.08043 ;v acc: 0.45
Epoch 4732;t loss: 2.92783 ;t acc: 0.64 ;v loss: 3.08040 ;v acc: 0.45
Epoch 4733;t loss: 2.92650 ;t acc: 0.65 ;v loss: 3.08035 ;v acc: 0.45
Epoch 4734;t loss: 2.92828 ;t acc: 0.64 ;v loss: 3.08034 ;v acc: 0.45
Epoch 4735;t loss: 2.92908 ;t acc: 0.65 ;v loss: 3.08031 ;v acc: 0.45
Epoch 4736;t loss: 2.93157 ;t acc: 0.64 ;v loss: 3.08033 ;v acc: 0.45
Epoch 4737;t loss: 2.92721 ;t acc: 0.64 ;v loss: 3.08031 ;v acc: 0.45
Epoch 4738;t loss: 2.92752 ;t acc: 0.64 ;v loss: 3.08027 ;v acc: 0.45
Epoch 4739;t loss: 2.92664 ;t acc: 0.64 ;v loss: 3.08025 ;v acc: 0.45
Epoch 4740;t loss: 2.92521 ;t acc: 0.64 ;v loss: 3.08022 ;v acc: 0.45
tensor([21,  8], device='cuda:0') tensor([21,  8], device='cuda:0')
tensor([ 2, 28], device='cuda:0') tensor([ 3, 28], device='cuda:0')
tensor([ 4, 12], device='cuda:0') tensor([31, 12], device='cuda:0')
tensor([10,  6], device='cuda:0') tensor([10,  6], device='cuda:0')
tensor([28, 10], device='cud

In [ ]:
model.load_state_dict(torch.load('./best_params'))
test_loader = DataLoader(test_dataset, batch_size=1)
loss, acc = test(train_loader, False,target=target)
print(f'Train loss: {loss} ; Train acc: {acc}')
loss, acc = test(val_loader, False,target=target)

print(f'Val loss: {loss} ; Val acc: {acc}')
loss, acc = test(test_loader, True,target=target)
print(f'Test loss: {loss} ; Test acc: {acc}')

# TODO: scheduler(?) Loss/acc records